In [1]:
from fairseq.models.roberta import RobertaModel
from scipy.stats import pearsonr

In [2]:
roberta = RobertaModel.from_pretrained(
    '/home/vv/',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/home/vv/dev/fairseq/STS-B-bin'
)

loading archive file /home/vv/
loading archive file /home/vv/dev/fairseq/STS-B-bin
| [input] dictionary: 50265 types


In [3]:
roberta.cuda()
roberta.eval()

RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=Tru

In [5]:
import pandas as pd
import os
import numpy as np
import requests
import json
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [6]:
fileDir

'/home/vv/dev/ai/experiments/sentence-similarity'

In [7]:
data1 = open('/home/vv/dev/ai/Benchmark/Sentence Similarity V.0.1/sts-dev.csv')
d1 = data1.readlines()
data2 = open('/home/vv/dev/ai/Benchmark/Sentence Similarity V.0.1/sts-test.csv')
d2 = data2.readlines()
data3 = open('/home/vv/dev/ai/Benchmark/Sentence Similarity V.0.1/sts-train.csv')
d3 = data3.readlines()
data = d1 + d2 + d3
print(len(data))

8628


In [8]:
df = []
for i in range(len(data)):
    df.append(data[i].split('\t'))
score = []
sentence1 = []
sentence2 = []
for j in range(len(df)):
    score.append(df[j][4])
    sentence1.append(df[j][5])
    b = df[j][6]
    sentence2.append(b.rstrip("\n"))

In [9]:
dict = list(zip(sentence1, sentence2, score))
df = pd.DataFrame(dict, columns = ['sentence1', 'sentence2', 'score'])

In [10]:
df.head()

sentence1  \
0     A man with a hard hat is dancing.   
1      A young child is riding a horse.   
2  A man is feeding a mouse to a snake.   
3        A woman is playing the guitar.   
4         A woman is playing the flute.   

                                  sentence2  score  
0      A man wearing a hard hat is dancing.  5.000  
1                A child is riding a horse.  4.750  
2  The man is feeding a mouse to the snake.  5.000  
3                  A man is playing guitar.  2.400  
4                 A man is playing a flute.  2.750

In [11]:
scaler = MinMaxScaler()
df['score'] = scaler.fit_transform(df['score'].values.reshape(-1,1))

In [14]:
r = []
for i in range(len(df)):
    sent1 = df.sentence1[i]
    sent2 = df.sentence2[i]
    tokens = roberta.encode(sent1, sent2)
    features = roberta.extract_features(tokens)
    predictions = 5.0 * roberta.model.classification_heads['sentence_classification_head'](features)
    r.append(predictions.item())
df["roberta"] = r

In [31]:
df.head(100)

sentence1  \
0              A man with a hard hat is dancing.   
1               A young child is riding a horse.   
2           A man is feeding a mouse to a snake.   
3                 A woman is playing the guitar.   
4                  A woman is playing the flute.   
..                                           ...   
95               The cook is kneading the flour.   
96  A woman is riding a horse along a perimeter.   
97   A woman is putting her baby in a waste bin.   
98            A cat is pouncing on a trampoline.   
99        A man and woman are eating at a table.   

                                   sentence2  score   roberta  
0       A man wearing a hard hat is dancing.   1.00  5.207114  
1                 A child is riding a horse.   0.95  4.685174  
2   The man is feeding a mouse to the snake.   1.00  5.184724  
3                   A man is playing guitar.   0.48  2.423417  
4                  A man is playing a flute.   0.55  2.680269  
..                                       ...    ...       ...  
95       A woman is kneading tortilla dough.   0.52  2.263538  
96             A dog is riding a skateboard.   0.12  0.157577  
97  The woman is poking holes in the potato.   0.00  0.206841  
98                A man is slicing a tomato.   0.00 -0.176202  
99   A couple are eating a meal and talking.   0.60  3.222274  

[100 rows x 4 columns]

In [30]:
import psycopg2
try:
    connection = psycopg2.connect(user="postgres",
                                  password="cx6ac54nmgGtLD1y",
                                  host="35.200.234.61",
                                  port="5432",
                                  database="sales")
    for i in range(len(df)):
        cursor = connection.cursor()
        postgres_insert_query = """ UPDATE public.sts_benchmark SET roberta_score = %s WHERE sentence1 = %s and sentence2 = %s and score=%s"""
        record_to_insert = (df.roberta[i], df.sentence1[i], df.sentence2[i], df.score[i])
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        count = cursor.rowcount
        print (i, "Record inserted successfully into mobile table")
except (Exception, psycopg2.Error) as error :
    if(connection):
        print("Failed to insert record into mobile table", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

0 Record inserted successfully into mobile table
1 Record inserted successfully into mobile table
2 Record inserted successfully into mobile table
3 Record inserted successfully into mobile table
4 Record inserted successfully into mobile table
5 Record inserted successfully into mobile table
6 Record inserted successfully into mobile table
7 Record inserted successfully into mobile table
8 Record inserted successfully into mobile table
9 Record inserted successfully into mobile table
10 Record inserted successfully into mobile table
11 Record inserted successfully into mobile table
12 Record inserted successfully into mobile table
13 Record inserted successfully into mobile table
14 Record inserted successfully into mobile table
15 Record inserted successfully into mobile table
16 Record inserted successfully into mobile table
17 Record inserted successfully into mobile table
18 Record inserted successfully into mobile table
19 Record inserted successfully into mobile table
20 Record 

164 Record inserted successfully into mobile table
165 Record inserted successfully into mobile table
166 Record inserted successfully into mobile table
167 Record inserted successfully into mobile table
168 Record inserted successfully into mobile table
169 Record inserted successfully into mobile table
170 Record inserted successfully into mobile table
171 Record inserted successfully into mobile table
172 Record inserted successfully into mobile table
173 Record inserted successfully into mobile table
174 Record inserted successfully into mobile table
175 Record inserted successfully into mobile table
176 Record inserted successfully into mobile table
177 Record inserted successfully into mobile table
178 Record inserted successfully into mobile table
179 Record inserted successfully into mobile table
180 Record inserted successfully into mobile table
181 Record inserted successfully into mobile table
182 Record inserted successfully into mobile table
183 Record inserted successfull

326 Record inserted successfully into mobile table
327 Record inserted successfully into mobile table
328 Record inserted successfully into mobile table
329 Record inserted successfully into mobile table
330 Record inserted successfully into mobile table
331 Record inserted successfully into mobile table
332 Record inserted successfully into mobile table
333 Record inserted successfully into mobile table
334 Record inserted successfully into mobile table
335 Record inserted successfully into mobile table
336 Record inserted successfully into mobile table
337 Record inserted successfully into mobile table
338 Record inserted successfully into mobile table
339 Record inserted successfully into mobile table
340 Record inserted successfully into mobile table
341 Record inserted successfully into mobile table
342 Record inserted successfully into mobile table
343 Record inserted successfully into mobile table
344 Record inserted successfully into mobile table
345 Record inserted successfull

488 Record inserted successfully into mobile table
489 Record inserted successfully into mobile table
490 Record inserted successfully into mobile table
491 Record inserted successfully into mobile table
492 Record inserted successfully into mobile table
493 Record inserted successfully into mobile table
494 Record inserted successfully into mobile table
495 Record inserted successfully into mobile table
496 Record inserted successfully into mobile table
497 Record inserted successfully into mobile table
498 Record inserted successfully into mobile table
499 Record inserted successfully into mobile table
500 Record inserted successfully into mobile table
501 Record inserted successfully into mobile table
502 Record inserted successfully into mobile table
503 Record inserted successfully into mobile table
504 Record inserted successfully into mobile table
505 Record inserted successfully into mobile table
506 Record inserted successfully into mobile table
507 Record inserted successfull

649 Record inserted successfully into mobile table
650 Record inserted successfully into mobile table
651 Record inserted successfully into mobile table
652 Record inserted successfully into mobile table
653 Record inserted successfully into mobile table
654 Record inserted successfully into mobile table
655 Record inserted successfully into mobile table
656 Record inserted successfully into mobile table
657 Record inserted successfully into mobile table
658 Record inserted successfully into mobile table
659 Record inserted successfully into mobile table
660 Record inserted successfully into mobile table
661 Record inserted successfully into mobile table
662 Record inserted successfully into mobile table
663 Record inserted successfully into mobile table
664 Record inserted successfully into mobile table
665 Record inserted successfully into mobile table
666 Record inserted successfully into mobile table
667 Record inserted successfully into mobile table
668 Record inserted successfull

...

7930 Record inserted successfully into mobile table
7931 Record inserted successfully into mobile table
7932 Record inserted successfully into mobile table
7933 Record inserted successfully into mobile table
7934 Record inserted successfully into mobile table
7935 Record inserted successfully into mobile table
7936 Record inserted successfully into mobile table
7937 Record inserted successfully into mobile table
7938 Record inserted successfully into mobile table
7939 Record inserted successfully into mobile table
7940 Record inserted successfully into mobile table
7941 Record inserted successfully into mobile table
7942 Record inserted successfully into mobile table
7943 Record inserted successfully into mobile table
7944 Record inserted successfully into mobile table
7945 Record inserted successfully into mobile table
7946 Record inserted successfully into mobile table
7947 Record inserted successfully into mobile table
7948 Record inserted successfully into mobile table
7949 Record 

8088 Record inserted successfully into mobile table
8089 Record inserted successfully into mobile table
8090 Record inserted successfully into mobile table
8091 Record inserted successfully into mobile table
8092 Record inserted successfully into mobile table
8093 Record inserted successfully into mobile table
8094 Record inserted successfully into mobile table
8095 Record inserted successfully into mobile table
8096 Record inserted successfully into mobile table
8097 Record inserted successfully into mobile table
8098 Record inserted successfully into mobile table
8099 Record inserted successfully into mobile table
8100 Record inserted successfully into mobile table
8101 Record inserted successfully into mobile table
8102 Record inserted successfully into mobile table
8103 Record inserted successfully into mobile table
8104 Record inserted successfully into mobile table
8105 Record inserted successfully into mobile table
8106 Record inserted successfully into mobile table
8107 Record 

8246 Record inserted successfully into mobile table
8247 Record inserted successfully into mobile table
8248 Record inserted successfully into mobile table
8249 Record inserted successfully into mobile table
8250 Record inserted successfully into mobile table
8251 Record inserted successfully into mobile table
8252 Record inserted successfully into mobile table
8253 Record inserted successfully into mobile table
8254 Record inserted successfully into mobile table
8255 Record inserted successfully into mobile table
8256 Record inserted successfully into mobile table
8257 Record inserted successfully into mobile table
8258 Record inserted successfully into mobile table
8259 Record inserted successfully into mobile table
8260 Record inserted successfully into mobile table
8261 Record inserted successfully into mobile table
8262 Record inserted successfully into mobile table
8263 Record inserted successfully into mobile table
8264 Record inserted successfully into mobile table
8265 Record 

8404 Record inserted successfully into mobile table
8405 Record inserted successfully into mobile table
8406 Record inserted successfully into mobile table
8407 Record inserted successfully into mobile table
8408 Record inserted successfully into mobile table
8409 Record inserted successfully into mobile table
8410 Record inserted successfully into mobile table
8411 Record inserted successfully into mobile table
8412 Record inserted successfully into mobile table
8413 Record inserted successfully into mobile table
8414 Record inserted successfully into mobile table
8415 Record inserted successfully into mobile table
8416 Record inserted successfully into mobile table
8417 Record inserted successfully into mobile table
8418 Record inserted successfully into mobile table
8419 Record inserted successfully into mobile table
8420 Record inserted successfully into mobile table
8421 Record inserted successfully into mobile table
8422 Record inserted successfully into mobile table
8423 Record 

8562 Record inserted successfully into mobile table
8563 Record inserted successfully into mobile table
8564 Record inserted successfully into mobile table
8565 Record inserted successfully into mobile table
8566 Record inserted successfully into mobile table
8567 Record inserted successfully into mobile table
8568 Record inserted successfully into mobile table
8569 Record inserted successfully into mobile table
8570 Record inserted successfully into mobile table
8571 Record inserted successfully into mobile table
8572 Record inserted successfully into mobile table
8573 Record inserted successfully into mobile table
8574 Record inserted successfully into mobile table
8575 Record inserted successfully into mobile table
8576 Record inserted successfully into mobile table
8577 Record inserted successfully into mobile table
8578 Record inserted successfully into mobile table
8579 Record inserted successfully into mobile table
8580 Record inserted successfully into mobile table
8581 Record 

In [32]:
pearson_coef, p_value = pearsonr(df["score"], df["roberta"])
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value)

Pearson Correlation Coefficient:  0.9673298925131408 and a P-value of: 0.0
